In [157]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from interval import interval
from src.get_data import get_clean_player_data

# Read in Ids as strings - Numeric deletes first zeros!

In [62]:
from os import listdir

data_path = "./data/season_prediction/"

# init storage
list_schedule, list_boxscore, obs_schedule, obs_boxscore = list(), list(), list(), list()
list_season_id = [['2014-15'], ['2015-16'], ['2016-17'], ['2017-18'], 
                  ['2018-19'], ['2019-20'], ['2020-21']]

# load and divide datafiles
for file in sorted(listdir(data_path)):
    
    if file.startswith("boxscores"):
        tmp = pd.read_csv(data_path + file)
        list_boxscore.append(tmp)
        obs_boxscore.append(tmp.shape[0])
    
    elif file.startswith("schedule"):
        tmp = pd.read_csv(data_path + file)
        list_schedule.append(tmp)
        obs_schedule.append(tmp.shape[0])
    
    else:
        raise ValueError("Neither boxscore nor schedule!")
    
    print(file + ' loaded')

# check check
assert len(list_schedule) == len(list_boxscore), 'Not the same amount of seasons!'

# transform to pandas dataframes
df_schedule, df_boxscores = pd.concat(list_schedule), pd.concat(list_boxscore)
print('\n' + str(df_schedule.shape[0]) + ' games!' )

boxscores_14_15.csv loaded
boxscores_15_16.csv loaded
boxscores_16_17.csv loaded
boxscores_17_18.csv loaded
boxscores_18_19.csv loaded
boxscores_19_20.csv loaded
boxscores_20_21.csv loaded
schedule_14_15.csv loaded
schedule_15_16.csv loaded
schedule_16_17.csv loaded
schedule_17_18.csv loaded
schedule_18_19.csv loaded
schedule_19_20.csv loaded
schedule_20_21.csv loaded

16578 games!


In [28]:
df_schedule

,Unnamed: 0,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,0,22014,1610612742,DAL,Dallas Mavericks,21400002,2014-10-28,DAL @ SAS,L,240,...,24,33,17,9,3,10,20,100,-1,1
1,1,22014,1610612759,SAS,San Antonio Spurs,21400002,2014-10-28,SAS vs. DAL,W,240,...,29,38,23,5,3,21,20,101,1,1
2,2,22014,1610612747,LAL,Los Angeles Lakers,21400003,2014-10-28,LAL vs. HOU,L,240,...,25,36,16,7,3,13,32,90,-18,1
3,3,22014,1610612745,HOU,Houston Rockets,21400003,2014-10-28,HOU @ LAL,W,240,...,33,47,22,7,3,14,30,108,18,1
4,4,22014,1610612740,NOP,New Orleans Pelicans,21400001,2014-10-28,NOP vs. ORL,W,240,...,36,62,20,10,17,9,17,101,17,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2155,2155,22020,1610612761,TOR,Toronto Raptors,22001079,2021-05-16,TOR vs. IND,L,240,...,32,48,21,8,3,12,12,113,-12,1
2156,2156,22020,1610612746,LAC,LA Clippers,22001074,2021-05-16,LAC @ OKC,L,240,...,28,44,17,8,3,3,14,112,-5,1
2157,2157,22020,1610612760,OKC,Oklahoma City Thunder,22001074,2021-05-16,OKC vs. LAC,W,240,...,40,54,20,1,12,15,11,117,5,1
2158,2158,22020,1610612747,LAL,Los Angeles Lakers,22001072,2021-05-16,LAL @ NOP,W,240,...,34,42,30,14,5,10,16,110,12,1


Real Plus Minus Rating as an example:
- Regress the Real Adjusted Plus Minus unto traditional statistics such as PTS, REB ...


In [348]:
from nba_api.stats.endpoints import playbyplayv2
call = playbyplayv2.PlayByPlayV2(game_id="0022001074", start_period=1, end_period=4)
data = pd.concat(call.get_data_frames())
data.drop(data.tail(1).index,inplace=True)

In [349]:
data

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,VIDEO_AVAILABLE_FLAG
1,0022001074,4.0,10.0,0.0,1.0,9:11 PM,12:00,Jump Ball Brown vs. Zubac: Tip to Maledon,None,None,...,Clippers,LAC,4.0,1630177.0,Theo Maledon,1.610613e+09,Oklahoma City,Thunder,OKC,1
2,0022001074,7.0,6.0,28.0,1.0,9:11 PM,11:54,None,None,Zubac Personal Take Foul (P1.T1) (L.Wood),...,Thunder,OKC,1.0,0.0,None,NaN,None,None,None,1
3,0022001074,9.0,8.0,0.0,1.0,9:11 PM,11:54,None,None,SUB: Ibaka FOR Zubac,...,Clippers,LAC,0.0,0.0,None,NaN,None,None,None,0
4,0022001074,11.0,1.0,78.0,1.0,9:11 PM,11:43,Pokusevski 9' Floating Jump Shot (2 PTS),None,None,...,None,None,0.0,0.0,None,NaN,None,None,None,1
5,0022001074,12.0,2.0,1.0,1.0,9:12 PM,11:28,None,None,MISS Ibaka 20' Jump Shot,...,None,None,0.0,0.0,None,NaN,None,None,None,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437,0022001074,597.0,4.0,0.0,4.0,11:09 PM,0:05,Brown REBOUND (Off:8 Def:10),None,None,...,None,None,0.0,0.0,None,NaN,None,None,None,1
438,0022001074,598.0,1.0,107.0,4.0,11:09 PM,0:05,Brown Tip Dunk Shot (24 PTS),None,None,...,None,None,0.0,0.0,None,NaN,None,None,None,1
439,0022001074,599.0,2.0,79.0,4.0,11:09 PM,0:00,None,None,MISS Kennard 33' 3PT Pullup Jump Shot,...,None,None,0.0,0.0,None,NaN,None,None,None,1
440,0022001074,600.0,4.0,0.0,4.0,11:09 PM,0:00,None,None,Clippers Rebound,...,None,None,0.0,0.0,None,NaN,None,None,None,0


In [350]:
data['time_real_tmp'] = data['PCTIMESTRING'].apply(lambda x: re.sub(string=x, repl="", pattern=":"))

In [357]:
np.abs(data['time_real_tmp'].astype(float) * (5 - data['PERIOD'].astype(float)) - 4800) # transform time

1         0.0
2       184.0
3       184.0
4       228.0
5       288.0
        ...  
437    4795.0
438    4795.0
439    4800.0
440    4800.0
441    4800.0
Length: 441, dtype: float64

In [328]:
from nba_api.stats.endpoints import gamerotation
call = gamerotation.GameRotation(game_id="0022001074")
df = pd.concat(call.get_data_frames())
starting_lineup = df[df['IN_TIME_REAL'] == 0]

In [329]:
starting_lineup

,GAME_ID,TEAM_ID,TEAM_CITY,TEAM_NAME,PERSON_ID,PLAYER_FIRST,PLAYER_LAST,IN_TIME_REAL,OUT_TIME_REAL,PLAYER_PTS,PT_DIFF,USG_PCT
4,0022001074,1610612746,LA Clippers,Clippers,201976,Patrick,Beverley,0.0,5530.0,4,-10.0,0.190
6,0022001074,1610612746,LA Clippers,Clippers,202335,Patrick,Patterson,0.0,7200.0,3,-9.0,0.138
8,0022001074,1610612746,LA Clippers,Clippers,202704,Reggie,Jackson,0.0,4120.0,6,-13.0,0.313
11,0022001074,1610612746,LA Clippers,Clippers,1627826,Ivica,Zubac,0.0,60.0,0,0.0,0.000
12,0022001074,1610612746,LA Clippers,Clippers,1628379,Luke,Kennard,0.0,6190.0,7,-9.0,0.200
2,0022001074,1610612760,Oklahoma City,Thunder,1629650,Moses,Brown,0.0,7200.0,8,9.0,0.214
8,0022001074,1610612760,Oklahoma City,Thunder,1629676,Isaiah,Roby,0.0,4510.0,6,10.0,0.235
13,0022001074,1610612760,Oklahoma City,Thunder,1630177,Theo,Maledon,0.0,5530.0,3,10.0,0.182
15,0022001074,1610612760,Oklahoma City,Thunder,1630197,Aleksej,Pokusevski,0.0,4510.0,10,10.0,0.294
19,0022001074,1610612760,Oklahoma City,Thunder,1630221,Josh,Hall,0.0,7200.0,3,9.0,0.107


In [323]:
data['HOME_PTS'] = data[data['EVENTMSGTYPE'] == 1]['SCORE'].apply(lambda x: x.split(' - ')[0])
data['AWAY_PTS'] = data[data['EVENTMSGTYPE'] == 1]['SCORE'].apply(lambda x: x.split(' - ')[1])

data['HOME_PLUS_MINUS'] = data['HOME_PTS'].astype(float) - data['AWAY_PTS'].astype(float)
data['PTS_CHANGE'] = np.abs(data[data['EVENTMSGTYPE'] == 1]['HOME_PLUS_MINUS'].diff())

In [324]:
data

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,VIDEO_AVAILABLE_FLAG,HOME_PTS,AWAY_PTS,HOME_PLUS_MINUS,PTS_CHANGE
0,0022001074,2.0,12.0,0.0,1.0,9:11 PM,12:00,None,Start of 1st Period (9:11 PM EST),None,...,None,NaN,None,None,None,0,NaN,NaN,NaN,NaN
1,0022001074,4.0,10.0,0.0,1.0,9:11 PM,12:00,Jump Ball Brown vs. Zubac: Tip to Maledon,None,None,...,Theo Maledon,1.610613e+09,Oklahoma City,Thunder,OKC,1,NaN,NaN,NaN,NaN
2,0022001074,7.0,6.0,28.0,1.0,9:11 PM,11:54,None,None,Zubac Personal Take Foul (P1.T1) (L.Wood),...,None,NaN,None,None,None,1,NaN,NaN,NaN,NaN
3,0022001074,9.0,8.0,0.0,1.0,9:11 PM,11:54,None,None,SUB: Ibaka FOR Zubac,...,None,NaN,None,None,None,0,NaN,NaN,NaN,NaN
4,0022001074,11.0,1.0,78.0,1.0,9:11 PM,11:43,Pokusevski 9' Floating Jump Shot (2 PTS),None,None,...,None,NaN,None,None,None,1,0,2,-2.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438,0022001074,598.0,1.0,107.0,4.0,11:09 PM,0:05,Brown Tip Dunk Shot (24 PTS),None,None,...,None,NaN,None,None,None,1,112,117,-5.0,2.0
439,0022001074,599.0,2.0,79.0,4.0,11:09 PM,0:00,None,None,MISS Kennard 33' 3PT Pullup Jump Shot,...,None,NaN,None,None,None,1,NaN,NaN,NaN,NaN
440,0022001074,600.0,4.0,0.0,4.0,11:09 PM,0:00,None,None,Clippers Rebound,...,None,NaN,None,None,None,0,NaN,NaN,NaN,NaN
441,0022001074,601.0,13.0,0.0,4.0,11:10 PM,0:00,None,End of 4th Period (11:10 PM EST),None,...,None,NaN,None,None,None,1,NaN,NaN,NaN,NaN


In [307]:
data[data['EVENTMSGTYPE'] == 1]

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,VIDEO_AVAILABLE_FLAG,HOME_PTS,AWAY_PTS
4,0022001074,11.0,1.0,78.0,1.0,9:11 PM,11:43,Pokusevski 9' Floating Jump Shot (2 PTS),None,None,...,0.0,0.0,None,NaN,None,None,None,1,0,2
7,0022001074,14.0,1.0,47.0,1.0,9:12 PM,11:10,Roby 8' Turnaround Jump Shot (2 PTS),None,None,...,0.0,0.0,None,NaN,None,None,None,1,0,4
13,0022001074,21.0,1.0,79.0,1.0,9:13 PM,10:25,Maledon 25' 3PT Pullup Jump Shot (3 PTS),None,None,...,0.0,0.0,None,NaN,None,None,None,1,0,7
14,0022001074,22.0,1.0,1.0,1.0,9:13 PM,10:06,None,None,Jackson 3PT Jump Shot (3 PTS) (Patterson 1 AST),...,0.0,0.0,None,NaN,None,None,None,1,3,7
26,0022001074,36.0,1.0,97.0,1.0,9:15 PM,8:45,Brown Tip Layup Shot (2 PTS),None,None,...,0.0,0.0,None,NaN,None,None,None,1,3,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418,0022001074,571.0,1.0,79.0,4.0,11:03 PM,1:58,None,None,Kennard 17' Pullup Jump Shot (13 PTS) (Scrubb ...,...,0.0,0.0,None,NaN,None,None,None,1,110,108
421,0022001074,576.0,1.0,5.0,4.0,11:07 PM,1:46,Hall 2' Layup (25 PTS),None,None,...,0.0,0.0,None,NaN,None,None,None,1,110,110
432,0022001074,591.0,1.0,7.0,4.0,11:08 PM,0:57,None,None,Patterson 2' Dunk (10 PTS) (Oturu 2 AST),...,0.0,0.0,None,NaN,None,None,None,1,112,112
433,0022001074,593.0,1.0,80.0,4.0,11:09 PM,0:48,Pokusevski 26' 3PT Step Back Jump Shot (29 PTS),None,None,...,0.0,0.0,None,NaN,None,None,None,1,112,115


In [303]:
string = data[data['EVENTMSGTYPE'] == 1]['SCORE'].values[0]
data['HOME_PTS'] = string.split(' - ')[0]
data['AWAY_PTS'] = string.split(' - ')[1]

In [279]:
data_subs = data[data['EVENTMSGTYPE'] == 8]
data_subs.reset_index(inplace=True)

In [280]:
mask_home = data_subs['HOMEDESCRIPTION'].apply(lambda x: x is None)
mask_away = data_subs['VISITORDESCRIPTION'].apply(lambda x: x is None)

data_subs.loc[mask_home, 'HOMEDESCRIPTION'] = ''
data_subs.loc[mask_away, 'VISITORDESCRIPTION'] = ''

data_subs['DESCRIPTION'] = data_subs['HOMEDESCRIPTION'] + data_subs['VISITORDESCRIPTION']

/Users/hoener/opt/anaconda3/envs/ds/lib/python3.9/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-280-7af4d5cda3fe>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_subs['DESCRIPTION'] = data_subs['HOMEDESCRIPTION'] + data_subs['VISITORDESCRIPTION']


In [286]:
data_subs

,index,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,...,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,VIDEO_AVAILABLE_FLAG,DESCRIPTION
0,3,0022001074,9.0,8.0,0.0,1.0,9:11 PM,11:54,,None,...,LAC,0.0,0.0,None,NaN,None,None,None,0,SUB: Ibaka FOR Zubac
1,46,0022001074,58.0,8.0,0.0,1.0,9:20 PM,6:12,,None,...,LAC,0.0,0.0,None,NaN,None,None,None,0,SUB: Oturu FOR Ibaka
2,53,0022001074,69.0,8.0,0.0,1.0,9:22 PM,5:08,,None,...,LAC,0.0,0.0,None,NaN,None,None,None,0,SUB: Scrubb FOR Jackson
3,58,0022001074,76.0,8.0,0.0,1.0,9:22 PM,4:29,SUB: Hoard FOR Pokusevski,None,...,OKC,0.0,0.0,None,NaN,None,None,None,0,SUB: Hoard FOR Pokusevski
4,59,0022001074,77.0,8.0,0.0,1.0,9:22 PM,4:29,SUB: Brown Jr. FOR Roby,None,...,OKC,0.0,0.0,None,NaN,None,None,None,0,SUB: Brown Jr. FOR Roby
5,71,0022001074,96.0,8.0,0.0,1.0,9:27 PM,2:47,,None,...,LAC,0.0,0.0,None,NaN,None,None,None,0,SUB: Mann FOR Beverley
6,72,0022001074,107.0,8.0,0.0,1.0,9:27 PM,2:47,SUB: Pokusevski FOR Maledon,None,...,OKC,0.0,0.0,None,NaN,None,None,None,0,SUB: Pokusevski FOR Maledon
7,81,0022001074,113.0,8.0,0.0,1.0,9:29 PM,1:41,,None,...,LAC,0.0,0.0,None,NaN,None,None,None,0,SUB: Ferrell FOR Kennard
8,127,0022001074,176.0,8.0,0.0,2.0,9:41 PM,8:34,SUB: Hall FOR Pokusevski,None,...,OKC,0.0,0.0,None,NaN,None,None,None,0,SUB: Hall FOR Pokusevski
9,144,0022001074,198.0,8.0,0.0,2.0,9:45 PM,7:09,SUB: Pokusevski FOR Hoard,None,...,OKC,0.0,0.0,None,NaN,None,None,None,0,SUB: Pokusevski FOR Hoard


# Sub in and out using PLAYER1_ID and PLAYER2_ID: SUB: Player2 FOR Player1

In [251]:
import re
def extract_substitution(data, col, i):
    
    # get player who is subbed in
    in_player = (data[col].values[i].split("SUB: "))[1].split(" FOR")[0]
    
    # get player who is subbed out
    out_player = re.sub(string=data[col].values[i], repl="", pattern=f'SUB: {in_player} FOR ')
    
    return [in_player, out_player]

In [252]:
extract_substitution(data=data_subs, col='HOMEDESCRIPTION', i = 3)

['Hoard', 'Pokusevski']

In [238]:
def find_between(s, start, end):
    return (s.split(start))[1].split(end)[0]

In [250]:
import re
in_player = find_between(data_subs['HOMEDESCRIPTION'].values[3], start="SUB: ", end=" FOR")
re.sub(string=data_subs['HOMEDESCRIPTION'].values[3], repl="", pattern=f'SUB: {in_player} FOR ')

'Pokusevski'

In [236]:
data_subs = data[data['EVENTMSGTYPE'] == 8]

In [237]:
data_subs

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,VIDEO_AVAILABLE_FLAG
3,0022001074,9.0,8.0,0.0,1.0,9:11 PM,11:54,None,None,SUB: Ibaka FOR Zubac,...,Clippers,LAC,0.0,0.0,None,NaN,None,None,None,0
46,0022001074,58.0,8.0,0.0,1.0,9:20 PM,6:12,None,None,SUB: Oturu FOR Ibaka,...,Clippers,LAC,0.0,0.0,None,NaN,None,None,None,0
53,0022001074,69.0,8.0,0.0,1.0,9:22 PM,5:08,None,None,SUB: Scrubb FOR Jackson,...,Clippers,LAC,0.0,0.0,None,NaN,None,None,None,0
58,0022001074,76.0,8.0,0.0,1.0,9:22 PM,4:29,SUB: Hoard FOR Pokusevski,None,None,...,Thunder,OKC,0.0,0.0,None,NaN,None,None,None,0
59,0022001074,77.0,8.0,0.0,1.0,9:22 PM,4:29,SUB: Brown Jr. FOR Roby,None,None,...,Thunder,OKC,0.0,0.0,None,NaN,None,None,None,0
71,0022001074,96.0,8.0,0.0,1.0,9:27 PM,2:47,None,None,SUB: Mann FOR Beverley,...,Clippers,LAC,0.0,0.0,None,NaN,None,None,None,0
72,0022001074,107.0,8.0,0.0,1.0,9:27 PM,2:47,SUB: Pokusevski FOR Maledon,None,None,...,Thunder,OKC,0.0,0.0,None,NaN,None,None,None,0
81,0022001074,113.0,8.0,0.0,1.0,9:29 PM,1:41,None,None,SUB: Ferrell FOR Kennard,...,Clippers,LAC,0.0,0.0,None,NaN,None,None,None,0
127,0022001074,176.0,8.0,0.0,2.0,9:41 PM,8:34,SUB: Hall FOR Pokusevski,None,None,...,Thunder,OKC,0.0,0.0,None,NaN,None,None,None,0
144,0022001074,198.0,8.0,0.0,2.0,9:45 PM,7:09,SUB: Pokusevski FOR Hoard,None,None,...,Thunder,OKC,0.0,0.0,None,NaN,None,None,None,0


In [224]:
data['HOMEDESCRIPTION'].to_string()

"0                                                   None\n1              Jump Ball Brown vs. Zubac: Tip to Maledon\n2                                                   None\n3                                                   None\n4               Pokusevski 9' Floating Jump Shot (2 PTS)\n5                                                   None\n6                             Roby REBOUND (Off:0 Def:1)\n7                   Roby 8' Turnaround Jump Shot (2 PTS)\n8                                    Brown BLOCK (1 BLK)\n9                             Hall REBOUND (Off:0 Def:1)\n10           Maledon Step Out of Bounds Turnover (P1.T1)\n11                                                  None\n12                           Brown REBOUND (Off:0 Def:1)\n13              Maledon 25' 3PT Pullup Jump Shot (3 PTS)\n14                                                  None\n15                 MISS Hall 8' Turnaround Fadeaway Shot\n16                                                  None\n17           

# Divide time by 600 to get minutes!

In [99]:
def get_substitutions(data: pd.DataFrame):
    return data['IN_TIME_REAL'].unique()

In [126]:
from nba_api.stats.endpoints import gamerotation
import pandas as pd
import numpy as np

class GetStints:
    
    def __init__(self, game_id: str):
        call = gamerotation.GameRotation(game_id=game_id)
        self.data = pd.concat(call.get_data_frames())
    
    def _get_substitutions(self):
        # get number of substitutions / unique In times
        return self.data['IN_TIME_REAL'].unique()

    def get_on_court(self, data, clock):

        # create bool object, all players subbed in at or after clock time and subbed out after clock time
        bool_court = (data['IN_TIME_REAL'] <= clock) & (data['OUT_TIME_REAL'] > clock)
        data_tmp = data[bool_court]
        
        first_subout = data_tmp['OUT_TIME_REAL'].min()
        

        # check check
        assert data_return.shape[0] == 10, "Not 10 players on the court - error!"
    
        # return subset data
        return data[bool_court]

    def get_stints(self):
        
        # setup the loop, get all times for in-substitutions
        clocks = self._get_substitutions()
        list_stints = list()
        
        # create stints indicator
        n_stints = clocks.shape[0]
        stint = np.arange(start=1, stop=1+n_stints)
        stints = np.repeat(stint, 10)
        
        # loop over these times and get the squad on the floor for each stint
        for t in clocks:
            list_stints.append(self.get_on_court(data=self.data, clock=t))
        
        # transform to dataframe and create number of stint indicator
        df = pd.concat(list_stints)
        df['stint'] = stints
        
        return df

In [142]:
def get_on_court(data: pd.DataFrame, clock: float):
    
    # create bool object, all players subbed in at or after clock time and subbed out after clock time
    bool_tmp = (data['IN_TIME_REAL'] <= clock) & (data['OUT_TIME_REAL'] > clock)
    data_return = data[bool_tmp]
    
    # check check
    assert data_return.shape[0] == 10, "Not 10 players on the court - error!"
    
    first_out = data_return['OUT_TIME_REAL'].min()
    
    # return subset data
    return data[bool_court], first_out

In [128]:
GetStints(game_id="0022001074").get_stints()

,GAME_ID,TEAM_ID,TEAM_CITY,TEAM_NAME,PERSON_ID,PLAYER_FIRST,PLAYER_LAST,IN_TIME_REAL,OUT_TIME_REAL,PLAYER_PTS,PT_DIFF,USG_PCT,stint
0,0022001074,1610612746,LA Clippers,Clippers,201586,Serge,Ibaka,60.0,3480.0,0,-10.0,0.231,1
4,0022001074,1610612746,LA Clippers,Clippers,201976,Patrick,Beverley,0.0,5530.0,4,-10.0,0.190,1
6,0022001074,1610612746,LA Clippers,Clippers,202335,Patrick,Patterson,0.0,7200.0,3,-9.0,0.138,1
8,0022001074,1610612746,LA Clippers,Clippers,202704,Reggie,Jackson,0.0,4120.0,6,-13.0,0.313,1
12,0022001074,1610612746,LA Clippers,Clippers,1628379,Luke,Kennard,0.0,6190.0,7,-9.0,0.200,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0022001074,1610612760,Oklahoma City,Thunder,1628396,Tony,Bradley,7200.0,10740.0,0,-5.0,0.063,23
5,0022001074,1610612760,Oklahoma City,Thunder,1629658,Jaylen,Hoard,4510.0,10110.0,4,-7.0,0.200,23
10,0022001074,1610612760,Oklahoma City,Thunder,1629718,Charlie,Brown Jr.,4510.0,11930.0,6,-9.0,0.212,23
20,0022001074,1610612760,Oklahoma City,Thunder,1630221,Josh,Hall,9260.0,20490.0,13,-7.0,0.356,23


In [130]:
data

,GAME_ID,TEAM_ID,TEAM_CITY,TEAM_NAME,PERSON_ID,PLAYER_FIRST,PLAYER_LAST,IN_TIME_REAL,OUT_TIME_REAL,PLAYER_PTS,PT_DIFF,USG_PCT
0,0022001074,1610612746,LA Clippers,Clippers,201586,Serge,Ibaka,60.0,3480.0,0,-10.0,0.231
1,0022001074,1610612746,LA Clippers,Clippers,201586,Serge,Ibaka,7200.0,10740.0,3,5.0,0.250
2,0022001074,1610612746,LA Clippers,Clippers,201586,Serge,Ibaka,14400.0,15900.0,7,8.0,0.429
3,0022001074,1610612746,LA Clippers,Clippers,201586,Serge,Ibaka,18800.0,21600.0,2,-4.0,0.250
4,0022001074,1610612746,LA Clippers,Clippers,201976,Patrick,Beverley,0.0,5530.0,4,-10.0,0.190
5,0022001074,1610612746,LA Clippers,Clippers,201976,Patrick,Beverley,10110.0,15900.0,7,9.0,0.200
6,0022001074,1610612746,LA Clippers,Clippers,202335,Patrick,Patterson,0.0,7200.0,3,-9.0,0.138
7,0022001074,1610612746,LA Clippers,Clippers,202335,Patrick,Patterson,10740.0,28800.0,7,-1.0,0.090
8,0022001074,1610612746,LA Clippers,Clippers,202704,Reggie,Jackson,0.0,4120.0,6,-13.0,0.313
9,0022001074,1610612746,LA Clippers,Clippers,1627812,Yogi,Ferrell,6190.0,11640.0,5,5.0,0.150


In [201]:
tmp = list()
tmp2 = list()

for i in np.arange(data.shape[0]):
    tmp.append(interval[data['IN_TIME_REAL'].values[i], data['OUT_TIME_REAL'].values[i]])
    tmp2.append(data['PERSON_ID'].values[i])

ids = np.repeat(tmp2, data.shape[0])
ids2 = tmp2 * data.shape[0]

In [189]:
from itertools import product
store = list()

for a, b in product(tmp, tmp):
    store.append(a & b)

In [204]:
test = pd.DataFrame({'PLAYER_ID':ids, 'PLAYER_MATE_ID':ids2, 'INTERVAL':store})

In [210]:
test[test['INTERVAL'] != ()]

,PLAYER_ID,PLAYER_MATE_ID,INTERVAL
0,201586,201586,"((60.0, 3480.0),)"
4,201586,201976,"((60.0, 3480.0),)"
6,201586,202335,"((60.0, 3480.0),)"
8,201586,202704,"((60.0, 3480.0),)"
11,201586,1627826,"((60.0, 60.0),)"
...,...,...,...
2297,1630466,1630197,"((18110.0, 18110.0),)"
2298,1630466,1630197,"((20490.0, 24600.0),)"
2300,1630466,1630221,"((18110.0, 20490.0),)"
2301,1630466,1630221,"((21600.0, 24600.0),)"


In [182]:
tmp[4]

interval([0.0, 5530.0])

In [183]:
tmp[6]

interval([0.0, 7200.0])

In [181]:
tmp[4] & tmp[6]

interval([0.0, 5530.0])

In [141]:
get_on_court(data=data, clock=60)

,GAME_ID,TEAM_ID,TEAM_CITY,TEAM_NAME,PERSON_ID,PLAYER_FIRST,PLAYER_LAST,IN_TIME_REAL,OUT_TIME_REAL,PLAYER_PTS,PT_DIFF,USG_PCT
0,0022001074,1610612746,LA Clippers,Clippers,201586,Serge,Ibaka,60.0,3480.0,0,-10.0,0.231
4,0022001074,1610612746,LA Clippers,Clippers,201976,Patrick,Beverley,0.0,5530.0,4,-10.0,0.190
6,0022001074,1610612746,LA Clippers,Clippers,202335,Patrick,Patterson,0.0,7200.0,3,-9.0,0.138
8,0022001074,1610612746,LA Clippers,Clippers,202704,Reggie,Jackson,0.0,4120.0,6,-13.0,0.313
12,0022001074,1610612746,LA Clippers,Clippers,1628379,Luke,Kennard,0.0,6190.0,7,-9.0,0.200
2,0022001074,1610612760,Oklahoma City,Thunder,1629650,Moses,Brown,0.0,7200.0,8,9.0,0.214
8,0022001074,1610612760,Oklahoma City,Thunder,1629676,Isaiah,Roby,0.0,4510.0,6,10.0,0.235
13,0022001074,1610612760,Oklahoma City,Thunder,1630177,Theo,Maledon,0.0,5530.0,3,10.0,0.182
15,0022001074,1610612760,Oklahoma City,Thunder,1630197,Aleksej,Pokusevski,0.0,4510.0,10,10.0,0.294
19,0022001074,1610612760,Oklahoma City,Thunder,1630221,Josh,Hall,0.0,7200.0,3,9.0,0.107


In [101]:
clocks = get_substitutions(data=data)

In [102]:
clocks.shape[0]

23

In [149]:
stint, first_out = get_on_court(data=data, clock=60)

In [150]:
stint2, first_out2 = get_on_court(data=data, clock=first_out)

In [151]:
first_out

3480.0

In [152]:
stint

,GAME_ID,TEAM_ID,TEAM_CITY,TEAM_NAME,PERSON_ID,PLAYER_FIRST,PLAYER_LAST,IN_TIME_REAL,OUT_TIME_REAL,PLAYER_PTS,PT_DIFF,USG_PCT
0,0022001074,1610612746,LA Clippers,Clippers,201586,Serge,Ibaka,60.0,3480.0,0,-10.0,0.231
4,0022001074,1610612746,LA Clippers,Clippers,201976,Patrick,Beverley,0.0,5530.0,4,-10.0,0.190
6,0022001074,1610612746,LA Clippers,Clippers,202335,Patrick,Patterson,0.0,7200.0,3,-9.0,0.138
8,0022001074,1610612746,LA Clippers,Clippers,202704,Reggie,Jackson,0.0,4120.0,6,-13.0,0.313
12,0022001074,1610612746,LA Clippers,Clippers,1628379,Luke,Kennard,0.0,6190.0,7,-9.0,0.200
2,0022001074,1610612760,Oklahoma City,Thunder,1629650,Moses,Brown,0.0,7200.0,8,9.0,0.214
8,0022001074,1610612760,Oklahoma City,Thunder,1629676,Isaiah,Roby,0.0,4510.0,6,10.0,0.235
13,0022001074,1610612760,Oklahoma City,Thunder,1630177,Theo,Maledon,0.0,5530.0,3,10.0,0.182
15,0022001074,1610612760,Oklahoma City,Thunder,1630197,Aleksej,Pokusevski,0.0,4510.0,10,10.0,0.294
19,0022001074,1610612760,Oklahoma City,Thunder,1630221,Josh,Hall,0.0,7200.0,3,9.0,0.107


In [153]:
stint2

,GAME_ID,TEAM_ID,TEAM_CITY,TEAM_NAME,PERSON_ID,PLAYER_FIRST,PLAYER_LAST,IN_TIME_REAL,OUT_TIME_REAL,PLAYER_PTS,PT_DIFF,USG_PCT
0,0022001074,1610612746,LA Clippers,Clippers,201586,Serge,Ibaka,60.0,3480.0,0,-10.0,0.231
4,0022001074,1610612746,LA Clippers,Clippers,201976,Patrick,Beverley,0.0,5530.0,4,-10.0,0.190
6,0022001074,1610612746,LA Clippers,Clippers,202335,Patrick,Patterson,0.0,7200.0,3,-9.0,0.138
8,0022001074,1610612746,LA Clippers,Clippers,202704,Reggie,Jackson,0.0,4120.0,6,-13.0,0.313
12,0022001074,1610612746,LA Clippers,Clippers,1628379,Luke,Kennard,0.0,6190.0,7,-9.0,0.200
2,0022001074,1610612760,Oklahoma City,Thunder,1629650,Moses,Brown,0.0,7200.0,8,9.0,0.214
8,0022001074,1610612760,Oklahoma City,Thunder,1629676,Isaiah,Roby,0.0,4510.0,6,10.0,0.235
13,0022001074,1610612760,Oklahoma City,Thunder,1630177,Theo,Maledon,0.0,5530.0,3,10.0,0.182
15,0022001074,1610612760,Oklahoma City,Thunder,1630197,Aleksej,Pokusevski,0.0,4510.0,10,10.0,0.294
19,0022001074,1610612760,Oklahoma City,Thunder,1630221,Josh,Hall,0.0,7200.0,3,9.0,0.107


In [80]:
from nba_api.stats.endpoints import gamerotation
call = gamerotation.GameRotation(game_id="0022001074")
data = pd.concat(call.get_data_frames())
data

,GAME_ID,TEAM_ID,TEAM_CITY,TEAM_NAME,PERSON_ID,PLAYER_FIRST,PLAYER_LAST,IN_TIME_REAL,OUT_TIME_REAL,PLAYER_PTS,PT_DIFF,USG_PCT
0,0022001074,1610612746,LA Clippers,Clippers,201586,Serge,Ibaka,60.0,3480.0,0,-10.0,0.231
1,0022001074,1610612746,LA Clippers,Clippers,201586,Serge,Ibaka,7200.0,10740.0,3,5.0,0.250
2,0022001074,1610612746,LA Clippers,Clippers,201586,Serge,Ibaka,14400.0,15900.0,7,8.0,0.429
3,0022001074,1610612746,LA Clippers,Clippers,201586,Serge,Ibaka,18800.0,21600.0,2,-4.0,0.250
4,0022001074,1610612746,LA Clippers,Clippers,201976,Patrick,Beverley,0.0,5530.0,4,-10.0,0.190
5,0022001074,1610612746,LA Clippers,Clippers,201976,Patrick,Beverley,10110.0,15900.0,7,9.0,0.200
6,0022001074,1610612746,LA Clippers,Clippers,202335,Patrick,Patterson,0.0,7200.0,3,-9.0,0.138
7,0022001074,1610612746,LA Clippers,Clippers,202335,Patrick,Patterson,10740.0,28800.0,7,-1.0,0.090
8,0022001074,1610612746,LA Clippers,Clippers,202704,Reggie,Jackson,0.0,4120.0,6,-13.0,0.313
9,0022001074,1610612746,LA Clippers,Clippers,1627812,Yogi,Ferrell,6190.0,11640.0,5,5.0,0.150


In [86]:
data['OUT_TIME_REAL'].max() / 600

48.0

In [91]:
bool_starter = (data['IN_TIME_REAL'] == 0)
data[bool_starter]

,GAME_ID,TEAM_ID,TEAM_CITY,TEAM_NAME,PERSON_ID,PLAYER_FIRST,PLAYER_LAST,IN_TIME_REAL,OUT_TIME_REAL,PLAYER_PTS,PT_DIFF,USG_PCT
4,0022001074,1610612746,LA Clippers,Clippers,201976,Patrick,Beverley,0.0,5530.0,4,-10.0,0.190
6,0022001074,1610612746,LA Clippers,Clippers,202335,Patrick,Patterson,0.0,7200.0,3,-9.0,0.138
8,0022001074,1610612746,LA Clippers,Clippers,202704,Reggie,Jackson,0.0,4120.0,6,-13.0,0.313
11,0022001074,1610612746,LA Clippers,Clippers,1627826,Ivica,Zubac,0.0,60.0,0,0.0,0.000
12,0022001074,1610612746,LA Clippers,Clippers,1628379,Luke,Kennard,0.0,6190.0,7,-9.0,0.200
2,0022001074,1610612760,Oklahoma City,Thunder,1629650,Moses,Brown,0.0,7200.0,8,9.0,0.214
8,0022001074,1610612760,Oklahoma City,Thunder,1629676,Isaiah,Roby,0.0,4510.0,6,10.0,0.235
13,0022001074,1610612760,Oklahoma City,Thunder,1630177,Theo,Maledon,0.0,5530.0,3,10.0,0.182
15,0022001074,1610612760,Oklahoma City,Thunder,1630197,Aleksej,Pokusevski,0.0,4510.0,10,10.0,0.294
19,0022001074,1610612760,Oklahoma City,Thunder,1630221,Josh,Hall,0.0,7200.0,3,9.0,0.107
